In [1]:
from nltk.corpus import stopwords
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as  plt
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
from tensorflow import keras

import tensorflow as tf
#fitting for gpu


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

df_clean = pd.read_csv("./drive/My Drive/total_data_withcommainput.csv")
#---------------------------------------------------------------------------------------
given_ingredient = list(df_clean.ingredients)[0:192]
#-----------------------------创建评估用词表--------------------------------------------
reference_recipe = list(df_clean.recipes)
reference_meteor = []
references = []
reference_list = reference_recipe[0:192]
for astr in reference_list:
    alist = astr.split(' ')[1:-1]
    reference_meteor.append(' '.join(alist))
    references.append([alist])
clean_summaries = list(df_clean.ingredients)
clean_texts = list(df_clean.recipes)
sp_dataset = tuple(clean_summaries)
en_dataset = tuple(clean_texts)
#将词语式数据转成ID式
def tokenizer(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(
        num_words=None, filters='', split=' ')
    lang_tokenizer.fit_on_texts(lang)#统计词频，生成词表
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')#在句子后面做padding
    return tensor, lang_tokenizer

def tokenizer1(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(
        num_words=None, filters='', split=',')
    lang_tokenizer.fit_on_texts(lang)#统计词频，生成词表
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')#在句子后面做padding
    return tensor, lang_tokenizer

input_tensor, input_tokenizer = tokenizer1(sp_dataset)
output_tensor, output_tokenizer = tokenizer(en_dataset)

def max_length(tensor):
    return max(len(t) for t in tensor)

max_length_input = max_length(input_tensor)
max_length_output = max_length(output_tensor)
print(max_length_input, max_length_output)

from sklearn.model_selection import train_test_split
input_train, input_eval, output_train, output_eval = train_test_split(input_tensor[192:], output_tensor[192:], test_size=0.01, shuffle=False)

batch_size = 64
epochs = 10
dataset = tf.data.Dataset.from_tensor_slices((input_train, output_train))
train_dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder= True)
train_dataset = train_dataset.shuffle(10000)
eval_dataset = tf.data.Dataset.from_tensor_slices((input_eval, output_eval))
eval_dataset = eval_dataset.repeat(epochs).batch(batch_size, drop_remainder= True)

for x, y in train_dataset.take(1):
    print(x.shape)
    print(y.shape)
    print(x)
    print(y)

embedding_units = 256
units = 256
#input_tokenizer.word_index是个字典
input_vocab_size = len(input_tokenizer.word_index) + 1
output_vocab_size = len(output_tokenizer.word_index) + 1
print(input_vocab_size,output_vocab_size)
#-------------------lstm-----------------------------------
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_units, encoding_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_units)
        #由于用attention，每步输出需要return_sequences = True
        self.gru = keras.layers.LSTM(self.encoding_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
        
        
    #hidden是初始化的隐含状态
    #这里hidden应该变成[hiddenstate,hiddenc]
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state, state_c = self.gru(x, initial_state = hidden)
        #output = keras.layers.Dropout(0.75)(output)
        return output, state, state_c
    
    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_size, self.encoding_units)),tf.zeros((self.batch_size, self.encoding_units))]
    
encoder = Encoder(input_vocab_size, embedding_units, units, batch_size)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden, sample_c = encoder.call(x, sample_hidden)

print('sample_output.shape: ', sample_output.shape)
print('sample_hidden.shape: ', sample_hidden.shape)
print('sample_memory.shape: ', sample_c.shape)


class BahdanauAttention(keras.Model):
    
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
        
    def call(self, decoder_hidden, encoder_outputs):
        #decoder_hidden.shape = (batch_size, units)
        #encoder_outputs.shape = (batch_size, length, units)
        decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)#decoder_hidden.shape = (batch_size, 1, units)
        #before V:tf.nn.tanh/shape: (batch_size, length, units)
        #after V:(batch_size, length, 1)
        score = self.V(
            tf.nn.tanh(
                self.W1(encoder_outputs) + self.W2(decoder_hidden_with_time_axis)))
        
        #shape: (batch_size, length, 1)
        attention_weights = tf.nn.softmax(score, axis = 1)
        #context_vector.shape:(batch_size, length, units)
        context_vector = attention_weights * encoder_outputs
        #context_vector.shape:(batch_size, units)
        context_vector = tf.reduce_sum(context_vector, axis = 1)#在length上求和
        
        return context_vector, attention_weights

attention_model = BahdanauAttention(units = 10)#units:经过W1之后的units个数，与batch_size, length, units里units不同
attention_results, attention_weights = attention_model.call(sample_hidden,sample_output)

print(attention_results.shape)
print(attention_weights.shape)

18 216
(64, 18)
(64, 216)
tf.Tensor(
[[    1     3    50 ...     0     0     0]
 [    1   174   246 ...     0     0     0]
 [    1     3    78 ...     0     0     0]
 ...
 [    1 11717   308 ...     0     0     0]
 [    1   614   161 ...     0     0     0]
 [    1   138   267 ...     0     0     0]], shape=(64, 18), dtype=int32)
tf.Tensor(
[[ 11  42  17 ...   0   0   0]
 [ 11  42  17 ...   0   0   0]
 [ 11  42  17 ...   0   0   0]
 ...
 [ 11 517 688 ...   0   0   0]
 [ 11  24 340 ...   0   0   0]
 [ 11 136  48 ...   0   0   0]], shape=(64, 216), dtype=int32)
15173 6756
sample_output.shape:  (64, 18, 256)
sample_hidden.shape:  (64, 256)
sample_memory.shape:  (64, 256)
(64, 256)
(64, 18, 1)


In [2]:
#---------------------------------lstm-----------------------------------------
class Decoder(keras.Model):
    def __init__(self, vocab_size, embedding_units, decoding_units, batch_size):
        super(Decoder, self).__init__()#调用父类（keras.Model）的构造函数
        self.batch_size = batch_size
        self.decoding_units = decoding_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_units)
        self.gru = keras.layers.LSTM(self.decoding_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
        
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.decoding_units)
        
        #x:当前步输入，hidden:前一步输出
    def call(self, x, hidden, encoding_outputs,encoding_state):
        #context_vector.shape: (bathc_size, units)
        context_vector, attention_weights = self.attention.call(hidden, encoding_outputs)
        #befor embedding:x.shape:(batch_size, 1)
        #after embedding:x.shape:(batch_size, 1, embedding_units)
        
        x = self.embedding(x)
        
        combined_x = tf.concat([tf.expand_dims(context_vector, 1),x], axis = -1)
        #output.shape: (batch_size, 1, decoding_units)
        #state.shape: (batch_size, decoding_units)
        output, state, state_c = self.gru(combined_x, initial_state = [hidden,encoding_state])
        #output = keras.layers.Dropout(0.5)(output)
        #output.shape: (batch_size, decoding_units)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        #output.shape: (batch_size, vocab_size)
        output = self.fc(output)
        
        return context_vector, output, state, attention_weights, state_c
decoder = Decoder(output_vocab_size, embedding_units, units, batch_size)

outputs = decoder.call(tf.random.uniform((batch_size,1)), sample_hidden, sample_output,sample_hidden)

context_vector, decoder_output, decoder_hidden, decoder_aw, state_c = outputs

print(decoder_output.shape)
print(decoder_hidden.shape)
print(decoder_aw.shape)

(64, 6756)
(64, 256)
(64, 18, 1)


In [3]:
optimizer = keras.optimizers.Adam()

loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')
#from_logits直接经过fc的输出没有经过softmax,如果经过softmax就设成False

def loss_function(real, pred):
    #输出里的padding不应该计算到损失函数中去
    mask = tf.math.logical_not(tf.math.equal(real, 0))#是padding时，mask取0
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

#lstm-----------------------------------------
@tf.function#加速cell
def train_step(inp, targ, encoding_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        encoding_outputs, encoding_hidden, encoding_c = encoder(inp, encoding_hidden)
        
        decoding_hidden = encoding_hidden
        decoding_state = encoding_c
        #eg: <start> I am here <end>
        #1. <start> -> I
        #2. I -> am
        #3. am -> here
        #4. here -> <end>
        for t in range(0, targ.shape[1]-1):
            decoding_input = tf.expand_dims(targ[:, t],1)
            context_vector, predictions, decoding_hidden, decoding_aw, decoding_state = decoder.call(decoding_input, decoding_hidden, encoding_outputs,decoding_state)
            loss += loss_function(targ[:, t+1],predictions)
            
    batch_loss = loss / int(targ.shape[0])
        
    variables = encoder.trainable_variables + decoder.trainable_variables
        
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
        
    return batch_loss  

#-------------------------------------------lstm-------------------------
def eval_step(inp, targ, encoding_hidden):
    loss = 0
    
    encoding_outputs, encoding_hidden, encoding_c = encoder(inp, encoding_hidden)
        
    decoding_hidden = encoding_hidden
    decoding_state = encoding_c
        #eg: <start> I am here <end>
        #1. <start> -> I
        #2. I -> am
        #3. am -> here
        #4. here -> <end>
    for t in range(0, targ.shape[1]-1):
        decoding_input = tf.expand_dims(targ[:, t],1)
        context_vector, predictions, decoding_hidden, decoding_aw, decoding_state = decoder.call(decoding_input, decoding_hidden, encoding_outputs, decoding_state)
        loss += loss_function(targ[:, t+1],predictions)
            
    batch_loss = loss / int(targ.shape[0])
    return batch_loss 

In [4]:
adict = {'training_loss':[], 'validation_loss':[]}
alist = []
import time
#epochs = 20
epochs = 10
# steps_per_epoch = len(input_tensor[192:18973]) // batch_size
# steps_per_epoch1 = len(input_tensor[18973:]) // batch_size
steps_per_epoch = len(input_train) // batch_size
steps_per_epoch1 = len(input_eval) // batch_size
for epoch in range(epochs):
    start = time.time()
    
    encoding_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    total_loss1 = 0
    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        
        batch_loss = train_step(inp, targ, encoding_hidden)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:4f}'.format(epoch+1, batch, batch_loss.numpy()))
    for (batch1, (inp, targ)) in enumerate(eval_dataset.take(steps_per_epoch1)):
        eval_loss = eval_step(inp, targ, encoding_hidden)
        total_loss1 += eval_loss
               
            
      
    print('Epoch {} Loss {:4f}'.format(epoch+1, total_loss / steps_per_epoch))
    print('Epoch {} Eval_Loss {:4f}'.format(epoch+1, total_loss1 / steps_per_epoch1))
    print('Time take for 1 epoch {} sec\n'.format(time.time() - start))
    adict['training_loss'].append(round(float(total_loss / steps_per_epoch),3))
    adict['validation_loss'].append(round(float(total_loss1 / steps_per_epoch1),3))

Epoch 1 Batch 0 Loss 11.677101
Epoch 1 Batch 100 Loss 7.741089
Epoch 1 Batch 200 Loss 8.309892
Epoch 1 Loss 7.446211
Epoch 1 Eval_Loss 7.230153
Time take for 1 epoch 403.50923013687134 sec

Epoch 2 Batch 0 Loss 5.353696
Epoch 2 Batch 100 Loss 6.701190
Epoch 2 Batch 200 Loss 5.528084
Epoch 2 Loss 5.872949
Epoch 2 Eval_Loss 5.461769
Time take for 1 epoch 142.57539463043213 sec

Epoch 3 Batch 0 Loss 5.433547
Epoch 3 Batch 100 Loss 4.819163
Epoch 3 Batch 200 Loss 4.444465
Epoch 3 Loss 4.569850
Epoch 3 Eval_Loss 4.724972
Time take for 1 epoch 143.83809971809387 sec

Epoch 4 Batch 0 Loss 4.252310
Epoch 4 Batch 100 Loss 4.482774
Epoch 4 Batch 200 Loss 4.016005
Epoch 4 Loss 4.069714
Epoch 4 Eval_Loss 4.352441
Time take for 1 epoch 144.27436590194702 sec

Epoch 5 Batch 0 Loss 3.946141
Epoch 5 Batch 100 Loss 3.741661
Epoch 5 Batch 200 Loss 3.282374
Epoch 5 Loss 3.732124
Epoch 5 Eval_Loss 4.147300
Time take for 1 epoch 143.62438082695007 sec

Epoch 6 Batch 0 Loss 3.446806
Epoch 6 Batch 100 Loss 4

In [5]:
df_clean = pd.read_csv("./drive/My Drive/total_data.csv")
#---------------------------------------------------------------------------------------
given_ingredient = list(df_clean.ingredients)[0:192]
#-----------------------------创建评估用词表--------------------------------------------
reference_recipe = list(df_clean.recipes)
reference_meteor = []
references = []
reference_list = reference_recipe[0:192]
for astr in reference_list:
    alist = astr.split(' ')[1:-1]
    reference_meteor.append(' '.join(alist))
    references.append([alist])
clean_summaries = list(df_clean.ingredients)
clean_texts = list(df_clean.recipes)
sp_dataset = tuple(clean_summaries)
en_dataset = tuple(clean_texts)
#将词语式数据转成ID式
def tokenizer(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(
        num_words=None, filters='', split=' ')
    lang_tokenizer.fit_on_texts(lang)#统计词频，生成词表
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')#在句子后面做padding
    return tensor, lang_tokenizer


input_tensor, input_tokenizer = tokenizer(sp_dataset)
output_tensor, output_tokenizer = tokenizer(en_dataset)

def max_length(tensor):
    return max(len(t) for t in tensor)

max_length_input = max_length(input_tensor)
max_length_output = max_length(output_tensor)
print(max_length_input, max_length_output)

from sklearn.model_selection import train_test_split
input_train, input_eval, output_train, output_eval = train_test_split(input_tensor[192:], output_tensor[192:], test_size=0.01, shuffle=False)

batch_size = 64
epochs = 20
dataset = tf.data.Dataset.from_tensor_slices((input_train, output_train))
train_dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder= True)
train_dataset = train_dataset.shuffle(10000)
eval_dataset = tf.data.Dataset.from_tensor_slices((input_eval, output_eval))
eval_dataset = eval_dataset.repeat(epochs).batch(batch_size, drop_remainder= True)

for x, y in train_dataset.take(1):
    print(x.shape)
    print(y.shape)
    print(x)
    print(y)

embedding_units = 256
units = 256
#input_tokenizer.word_index是个字典
input_vocab_size = len(input_tokenizer.word_index) + 1
output_vocab_size = len(output_tokenizer.word_index) + 1
print(input_vocab_size,output_vocab_size)

35 216
(64, 35)
(64, 216)
tf.Tensor(
[[   1   99   15 ...    0    0    0]
 [   1   18    6 ...    0    0    0]
 [   1  130 1079 ...    0    0    0]
 ...
 [   1  448  160 ...    0    0    0]
 [   1   71   90 ...    0    0    0]
 [   1   16 1327 ...    0    0    0]], shape=(64, 35), dtype=int32)
tf.Tensor(
[[  11  634   50 ...    0    0    0]
 [  11   78   41 ...    0    0    0]
 [  11   28 1173 ...    0    0    0]
 ...
 [  11   28 1342 ...    0    0    0]
 [  11  240    5 ...    0    0    0]
 [  11   28  138 ...    0    0    0]], shape=(64, 216), dtype=int32)
3492 6728


In [6]:
#-------------------lstm-----------------------------------
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_units, encoding_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_units)
        #由于用attention，每步输出需要return_sequences = True
        self.gru = keras.layers.LSTM(self.encoding_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
        
        
    #hidden是初始化的隐含状态
    #这里hidden应该变成[hiddenstate,hiddenc]
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state, state_c = self.gru(x, initial_state = hidden)
        #output = keras.layers.Dropout(0.75)(output)
        return output, state, state_c
    
    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_size, self.encoding_units)),tf.zeros((self.batch_size, self.encoding_units))]
    
encoder = Encoder(input_vocab_size, embedding_units, units, batch_size)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden, sample_c = encoder.call(x, sample_hidden)
class BahdanauAttention(keras.Model):
    
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
        
    def call(self, decoder_hidden, encoder_outputs):
        #decoder_hidden.shape = (batch_size, units)
        #encoder_outputs.shape = (batch_size, length, units)
        decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)#decoder_hidden.shape = (batch_size, 1, units)
        #before V:tf.nn.tanh/shape: (batch_size, length, units)
        #after V:(batch_size, length, 1)
        score = self.V(
            tf.nn.tanh(
                self.W1(encoder_outputs) + self.W2(decoder_hidden_with_time_axis)))
        
        #shape: (batch_size, length, 1)
        attention_weights = tf.nn.softmax(score, axis = 1)
        #context_vector.shape:(batch_size, length, units)
        context_vector = attention_weights * encoder_outputs
        #context_vector.shape:(batch_size, units)
        context_vector = tf.reduce_sum(context_vector, axis = 1)#在length上求和
        
        return context_vector, attention_weights

attention_model = BahdanauAttention(units = 10)#units:经过W1之后的units个数，与batch_size, length, units里units不同
attention_results, attention_weights = attention_model.call(sample_hidden,sample_output)

print(attention_results.shape)
print(attention_weights.shape)
optimizer = keras.optimizers.Adam()

loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')
#from_logits直接经过fc的输出没有经过softmax,如果经过softmax就设成False

def loss_function(real, pred):
    #输出里的padding不应该计算到损失函数中去
    mask = tf.math.logical_not(tf.math.equal(real, 0))#是padding时，mask取0
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

W11 = keras.layers.Dense(1)
W22 = keras.layers.Dense(1)
W33 = keras.layers.Dense(1)


#lstm-----------------------------------------
#@tf.function#加速cell
def train_step(inp, targ, encoding_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        encoding_outputs, encoding_hidden, encoding_c = encoder(inp, encoding_hidden)
        inp = inp.numpy()
        decoding_hidden = encoding_hidden
        decoding_state = encoding_c
        #eg: <start> I am here <end>
        #1. <start> -> I
        #2. I -> am
        #3. am -> here
        #4. here -> <end>
        for t in range(0, targ.shape[1]-1):
            decoding_input = tf.expand_dims(targ[:, t],1)
            context_vector, predictions, decoding_hidden, decoding_aw, decoding_state = decoder.call(decoding_input, decoding_hidden, encoding_outputs,decoding_state)
            #由decoding.call返回的三维decoding_aw
            attention_weights = np.reshape(decoding_aw, (-1, 35))
            adpre = np.zeros((64,6756))
            
            for index1,sen in enumerate(inp):
                for index2, word_index in enumerate(sen):
                #int means only integer can be used as indices
                    adpre[index1][int(word_index)] = attention_weights[index1][index2]
            #tf.nn.sigmoid之前shape应该是64,1
            prob = tf.nn.sigmoid(W11(context_vector) + W22(decoding_hidden) + W33(decoding_input))
            #print(prob.shape)
            adpre = tf.convert_to_tensor(adpre,tf.float32)
            #prob = tf.conver_to_tensor(prob)
            #print(prob)
            #print(adpre)
            new_prediction = (1-prob) * adpre + prob * predictions
            loss += loss_function(targ[:, t+1],new_prediction)
            
    batch_loss = loss / int(targ.shape[0])
        
    variables = encoder.trainable_variables + decoder.trainable_variables + W11.trainable_variables + W22.trainable_variables + W33.trainable_variables
        
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
        
    return batch_loss   

#-------------------------------------------lstm-------------------------
def eval_step(inp, targ, encoding_hidden):
    loss = 0
    
    encoding_outputs, encoding_hidden, encoding_c = encoder(inp, encoding_hidden)
    inp = inp.numpy()
    decoding_hidden = encoding_hidden
    decoding_state = encoding_c
        #eg: <start> I am here <end>
        #1. <start> -> I
        #2. I -> am
        #3. am -> here
        #4. here -> <end>
    for t in range(0, targ.shape[1]-1):
        decoding_input = tf.expand_dims(targ[:, t],1)
        context_vector, predictions, decoding_hidden, decoding_aw, decoding_state = decoder.call(decoding_input, decoding_hidden, encoding_outputs, decoding_state)
        attention_weights = np.reshape(decoding_aw, (-1, 35))
        adpre = np.zeros((64,6756))
        for index1,sen in enumerate(inp):
            for index2, word_index in enumerate(sen):
                #int means only integer can be used as indices
                adpre[index1][int(word_index)] = attention_weights[index1][index2]
            #tf.nn.sigmoid之前shape应该是64,1
        prob = tf.nn.sigmoid(W11(context_vector) + W22(decoding_hidden) + W33(decoding_input))
        #print(prob.shape)
        adpre = tf.convert_to_tensor(adpre,tf.float32)
        #prob = tf.conver_to_tensor(prob)
        new_prediction = (1-prob) * adpre + prob * predictions
        loss += loss_function(targ[:, t+1],new_prediction)
            
    batch_loss = loss / int(targ.shape[0])
    return batch_loss 

(64, 256)
(64, 35, 1)


In [7]:
adict = {'training_loss': [], 'validation_loss': []}
alist = []
import time

# epochs = 20
epochs = 20
# steps_per_epoch = len(input_tensor[192:18973]) // batch_size
# steps_per_epoch1 = len(input_tensor[18973:]) // batch_size
steps_per_epoch = len(input_train) // batch_size
steps_per_epoch1 = len(input_eval) // batch_size
for epoch in range(epochs):
    start = time.time()

    encoding_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    total_loss1 = 0
    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        # inp_np = inp.numpy()
        # print(type(inp_np))
        batch_loss = train_step(inp, targ, encoding_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:4f}'.format(epoch + 1, batch, batch_loss.numpy()))


    for (batch1, (inp, targ)) in enumerate(eval_dataset.take(steps_per_epoch1)):
        eval_loss = eval_step(inp, targ, encoding_hidden)
        total_loss1 += eval_loss

    print('Epoch {} Loss {:4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Epoch {} Eval_Loss {:4f}'.format(epoch + 1, total_loss1 / steps_per_epoch1))
    print('Time take for 1 epoch {} sec\n'.format(time.time() - start))
    adict['training_loss'].append(round(float(total_loss / steps_per_epoch), 3))
    adict['validation_loss'].append(round(float(total_loss1 / steps_per_epoch1), 3))

Epoch 1 Batch 0 Loss 11.521899
Epoch 1 Batch 100 Loss 7.631863
Epoch 1 Batch 200 Loss 6.848181
Epoch 1 Loss 7.724232
Epoch 1 Eval_Loss 7.307006
Time take for 1 epoch 1430.0186970233917 sec

Epoch 2 Batch 0 Loss 6.547415
Epoch 2 Batch 100 Loss 5.890257
Epoch 2 Batch 200 Loss 5.910958
Epoch 2 Loss 6.092474
Epoch 2 Eval_Loss 6.222269
Time take for 1 epoch 1425.8674478530884 sec

Epoch 3 Batch 0 Loss 5.928827
Epoch 3 Batch 100 Loss 5.393955
Epoch 3 Batch 200 Loss 4.959885
Epoch 3 Loss 5.106209
Epoch 3 Eval_Loss 5.339112
Time take for 1 epoch 1428.2588880062103 sec

Epoch 4 Batch 0 Loss 4.372694
Epoch 4 Batch 100 Loss 4.475993
Epoch 4 Batch 200 Loss 3.737356
Epoch 4 Loss 3.749113
Epoch 4 Eval_Loss 3.917392
Time take for 1 epoch 1426.2309913635254 sec

Epoch 5 Batch 0 Loss 3.578132
Epoch 5 Batch 100 Loss 3.212610
Epoch 5 Batch 200 Loss 3.744883
Epoch 5 Loss 3.086846
Epoch 5 Eval_Loss 3.725421
Time take for 1 epoch 1427.5710146427155 sec

Epoch 6 Batch 0 Loss 3.073684
Epoch 6 Batch 100 Loss 3

In [8]:
print(adict)

{'training_loss': [7.724, 6.092, 5.106, 3.749, 3.087, 2.969, 2.88, 2.817, 2.717, 2.653, 2.621, 2.56, 2.526, 2.446, 2.35, 2.353, 2.315, 2.258, 2.217, 2.175], 'validation_loss': [7.307, 6.222, 5.339, 3.917, 3.725, 3.639, 3.572, 3.53, 3.489, 3.46, 3.426, 3.388, 3.387, 3.366, 3.355, 3.34, 3.329, 3.322, 3.344, 3.324]}


In [9]:
#Caution: 在不画图的时候一定要将下面两行代码注释掉
import matplotlib.ticker as ticker
#-------------------lstm----------------
def evaluate(input_sentence):
    attention_matrix = np.zeros((max_length_output, max_length_input))
    
    
    inputs = [input_tokenizer.word_index[token] for token in input_sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences(
    [inputs], maxlen=max_length_input, padding='post')
    
    inputs = tf.convert_to_tensor(inputs)
    results = ''
    #encoding_hidden = encoder.initialize_hidden_state()
    #这里的encoding_hidden在lstm情况下需要改一下维度
    encoding_hidden = [tf.zeros((1,units)),tf.zeros((1,units))]
    
    encoding_outputs, encoding_hidden, encoding_c = encoder(inputs, encoding_hidden)
    decoding_hidden = encoding_hidden
    decoding_state = encoding_c
    #eg: <start> I am here <end>
        #1. <start> -> I
        #2. I -> am
        #3. am -> here
        #4. here -> <end>
    #decoding_inpu.shape :(1,1)(batch_size, length)
    decoding_input = tf.expand_dims([output_tokenizer.word_index['<start>']],0)
    for t in range(max_length_output):
        #attention_weights.shape: (batch_size, input_length, 1)(1,16,1)
        context_vector, predictions, decoding_hidden, decoding_aw, decoding_state = decoder(decoding_input, decoding_hidden, encoding_outputs, decoding_state)
        
         #-------------------------------------------------------------------------
        attention_weights = np.reshape(decoding_aw, (-1, 35))
        #经历过上一步后attention_weights.shape = 1,35
        adpre = np.zeros((1,6756))
        for index1,sen in enumerate(inputs):
            for index2, word_index in enumerate(sen):
                #int means only integer can be used as indices
                adpre[index1][int(word_index)] = attention_weights[index1][index2]
            #tf.nn.sigmoid之前shape应该是64,1
        prob = tf.nn.sigmoid(W11(context_vector) + W22(decoding_hidden) + W33(decoding_input))
        #print(prob.shape)
        adpre = tf.convert_to_tensor(adpre,tf.float32)
        #prob = tf.conver_to_tensor(prob)
        new_prediction = (1-prob) * adpre + prob * predictions
        #-------------------------------------------------------------------------------
        
        attention_weights = tf.reshape(decoding_aw, (-1,))#length=16的向量
        attention_matrix[t] = attention_weights.numpy()#attention_weights是一个tensor，用numpy（）取出它的值
        #predictions.shape: (batch_size, vocab_size)  (1,4935)
        
        predicted_id = tf.argmax(new_prediction[0]).numpy()
        
        results += output_tokenizer.index_word[predicted_id] + ' '
        if output_tokenizer.index_word[predicted_id] == '<end>':
            return results, input_sentence, attention_matrix
        
        decoding_input = tf.expand_dims([predicted_id],0)
    return results, input_sentence, attention_matrix

def plot_attention(attention_matrix, input_sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1,1,1)#add a subplot,1,1,1表示子图位置
    ax.matshow(attention_matrix, cmap='viridis')#viridis是一种配色方案
    
    font_dict = {'fontsize': 14}
    ax.set_xticklabels([''] + input_sentence, fontdict=font_dict, rotation = 90)#seq2seq里不需要加空格
    ax.set_yticklabels([''] + predicted_sentence, fontdict=font_dict)
    plt.show()

def translate(input_sentence):
    results, input_sentence, attention_matrix = evaluate(input_sentence)
    
    #------------------------
    
    print('Input: %s' % (input_sentence))
    print('Predicted translation: %s' % (results))
    
    #attention_matrix = attention_matrix[:len(results.split(' ')), :len(input_sentence.split(' '))]
    
    #plot_attention(attention_matrix, input_sentence.split(' '), results.split(' '))
    return results


#----------------------BLEU and Meteor------------------------------
candidates = []
candidates_meteor = []
for sample1 in clean_summaries[0:192]:
    #res = translate(sample1+', ')
    res = translate(sample1)
    candidates_meteor.append(res)
    candidates.append(res.split(' '))

Input: <start> skinless boneless chicken breast butter condensed cream chicken soup onion finely diced refrigerated biscuit dough torn pieces <end>
Predicted translation: preheat oven to 350 degrees f 175 degrees c . place chicken breasts in a 9x13 inch baking dish . in a large skillet over medium heat , melt butter and stir in chicken . pour in soup and water . mix well and pour into a 9x13 inch baking dish . bake in the preheated oven for 45 minutes . remove from oven and let stand for 5 minutes . remove and discard . place the chicken pieces in a 9x13 inch baking dish . bake in the preheated oven for 45 minutes . <end> 
Input: <start> condensed cream mushroom soup package dry onion soup mix water pot roast <end>
Predicted translation: preheat oven to 350 degrees f 175 degrees c . in a medium bowl , mix together cream of mushroom soup , onion soup mix , water , and water . mix well and pour into a 9x13 inch baking dish . bake in preheated oven for 1 hour , or until the internal tempe

In [10]:
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(references, candidates,weights=(1,0,0,0))
score1 = corpus_bleu(references, candidates,weights=(0.33,0.33,0.33,0))
score2 = corpus_bleu(references, candidates,weights=(0.25,0.25,0.25,0.25))
print(score,score1,score2)

0.48709072383233104 0.2538592218898033 0.191611688034733


In [11]:
def calculate_given_item(given_ingredient, generated_recipe):
    '''
    given_ingredient = ['egg tomatoes','chicken']
    splitted_ingredient = [['egg','tomatoes'],['chicken']]
    generated_recipe = [['use','egg'],['use','chicken'],...]
    '''

    total_score = 0
    extra_score = 0
    splitted_ingredient = []
    for bstr in given_ingredient:
        splitted_ingredient.append(bstr.split(' '))

    for index, each_sample in enumerate(splitted_ingredient):
        # count1 covered items
        # count2 extra items
        count1 = 0
        count2 = 0
        for each_word in each_sample:
            # generated_recipe就是下面的candidates=[['use','egg'],['use','chicken'],...]
            if each_word in generated_recipe[index]:
                count1 += 1
            else:
                count2 += 1
        extra_score += count2 / len(each_sample)
        total_score += count1 / len(each_sample)
    # -------------------------------------------------------------------------
    for index, each_recipe in enumerate(generated_recipe):
        # count1 covered items
        # count2 extra items
        extra_item = 0
        for each_word in each_recipe:
            # generated_recipe就是下面的candidates=[['use','egg'],['use','chicken'],...]
            if each_word in input_tokenizer.word_index and each_word not in splitted_ingredient[index]:
                extra_item += 1

    # --------------------------------------------------------------------------

    total_score = total_score / len(given_ingredient)

    return total_score, extra_item

cover_item, extra_item = calculate_given_item(given_ingredient, candidates)
print(cover_item,extra_item)

0.4659481750126013 15


In [12]:
print(reference_meteor)

['place the chicken , butter , soup , and onion in a slow cooker , and fill with enough water to cover . cover , and cook for 5 to 6 hours on high . about 30 minutes before serving , place the torn biscuit dough in the slow cooker . cook until the dough is no longer raw in the center .', 'in a slow cooker , mix cream of mushroom soup , dry onion soup mix and water . place pot roast in slow cooker and coat with soup mixture . cook on high setting for 3 to 4 hours , or on low setting for 8 to 9 hours .', 'preheat oven to 350 degrees f 175 degrees c . lightly grease a 9x5 inch loaf pan . in a large bowl , combine flour , baking soda and salt . in a separate bowl , cream together butter and brown sugar . stir in eggs and mashed bananas until well blended . stir banana mixture into flour mixture stir just to moisten . pour batter into prepared loaf pan . bake in preheated oven for 60 to 65 minutes , until a toothpick inserted into center of the loaf comes out clean . let bread cool in pan f

In [13]:
print(candidates_meteor)

['preheat oven to 350 degrees f 175 degrees c . place chicken breasts in a 9x13 inch baking dish . in a large skillet over medium heat , melt butter and stir in chicken . pour in soup and water . mix well and pour into a 9x13 inch baking dish . bake in the preheated oven for 45 minutes . remove from oven and let stand for 5 minutes . remove and discard . place the chicken pieces in a 9x13 inch baking dish . bake in the preheated oven for 45 minutes . <end> ', 'preheat oven to 350 degrees f 175 degrees c . in a medium bowl , mix together cream of mushroom soup , onion soup mix , water , and water . mix well and pour into a 9x13 inch baking dish . bake in preheated oven for 1 hour , or until the internal temperature of the meat has reached 160 degrees f 70 degrees c . <end> ', 'preheat oven to 350 degrees f 175 degrees c . grease a 9x13 inch baking pan . in a large bowl , stir together the flour , baking soda , and salt . set aside . in a large bowl , cream together the butter and brown 